In [59]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import utils
import matplotlib.pyplot as plt
import pickle
import h5py

In [60]:
batch_size = 64
test_batch_size = 1024
epochs = 10
lr = 0.05
momentum = 0.5
seed = 777
log_interval = 200
noised = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [71]:
# From https://wjddyd66.github.io/pytorch/Pytorch-AutoEncoder/
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        dropout = torch.nn.Dropout(p=0.5)
        self.layer1 = nn.Sequential(
            nn.Linear(6714, 4096),  # batch x 16 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 2048),  # batch x 32 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 1024),  # batch x 32 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(1024)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(1024, 512),  # batch x 16 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),  # batch x 32 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),  # batch x 32 x 28 x 28
            nn.ReLU()
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size, -1)
        return out

In [73]:
# From https://wjddyd66.github.io/pytorch/Pytorch-AutoEncoder/
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        dropout = torch.nn.Dropout(p=0.5)
        self.layer1 = nn.Sequential(
            nn.Linear(128, 256),  # batch x 16 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),  # batch x 32 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(512),
            nn.Linear(512, 1024),  # batch x 32 x 28 x 28
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(1024, 2048),  # batch x 16 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 4096),  # batch x 32 x 28 x 28
            nn.ReLU(),
            dropout,
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 6714),  # batch x 32 x 28 x 28
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        return out

In [74]:
class CursinePredictDataset(Dataset):
    def __init__(self, file_path, transform=None):
        h5f = h5py.File(file_path, "r")
        self.ingredients = h5f['features_one_hot'][:]
        self.cursine = h5f['labels_int_enc'][:]
        h5f.close()
        self.transform = transform
        
    def __len__(self):
        return len(self.cursine)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ingredients = torch.FloatTensor(self.ingredients[idx])
        cursine = torch.FloatTensor([self.cursine[idx]])
        sample = (ingredients, cursine)

        if self.transform:
            sample = self.transform(sample)

        return sample

In [75]:
cursine_dataset = CursinePredictDataset("./Container/train")

for i in range(len(cursine_dataset)):
    sample = cursine_dataset[i]
    print(i, sample[0][:10], sample[1])
    if i == 5:
        break

0 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([7.])
1 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([0.])
2 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([8.])
3 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([8.])
4 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([14.])
5 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([9.])


In [77]:
def random_to_deterministic(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


transform = transforms.Compose([transforms.ToTensor()])

trainset = CursinePredictDataset("./Container/train")
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

#testset = datasets.MNIST(root='./datasets', train=False, download=True, transform=transform)
#test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)


encoder = Encoder().to(device)
decoder = Decoder().to(device)

parameters = list(encoder.parameters()) + list(decoder.parameters())

optimizer = optim.SGD(parameters, lr=lr, momentum=momentum)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9999)


def train(log_interval, encoder, decoder, device, train_loader, optimizer, criterion, epoch, noised):
    encoder.train()
    decoder.train()
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = encoder(data)
        output = decoder(output)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
        scheduler.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return data, output


def test(log_interval, encoder, decoder, device, test_loader, criterion):
    encoder.eval()
    decoder.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            output = encoder(data)
            output = decoder(output)
            test_loss += criterion(output, data).item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}\n'.format
          (test_loss, ))

    return data, output


for epoch in range(1, 11):
    train_data, train_output = train(log_interval, encoder, decoder, device, train_loader, optimizer, criterion, epoch, noised)
    #test_data, test_output = test(log_interval, encoder, decoder, device, test_loader, criterion)

torch.save([encoder, decoder], './models/Feature_AE_dropout_model')

#encoder, decoder = torch.load('./models/MNIST_AE.pkl')

# Check with test set
"""
for data, _ in test_loader:
    trans = transforms.RandomApply([AddGaussianNoise()], p=1)
    data = trans(data.to(device))
    test_output = decoder(encoder(data))
    test_data = data * 0.5 + 0.5
    test_output = test_output * 0.5 + 0.5
    out_img = torch.squeeze(test_output.cpu().data)
    print(out_img.size())
    for i in range(2):
        plt.subplot(1,2,1)
        plt.imshow(torch.squeeze(test_data[i]).cpu().numpy(),cmap='gray')
        plt.subplot(1,2,2)
        plt.imshow(out_img[i].numpy(),cmap='gray')
        plt.show()
"""

Train Epoch: 1 [0/23547 (0%)]	Loss: 0.336041
Train Epoch: 1 [12800/23547 (54%)]	Loss: 0.328116
Train Epoch: 2 [0/23547 (0%)]	Loss: 0.320510
Train Epoch: 2 [12800/23547 (54%)]	Loss: 0.314849
Train Epoch: 3 [0/23547 (0%)]	Loss: 0.308753
Train Epoch: 3 [12800/23547 (54%)]	Loss: 0.303538
Train Epoch: 4 [0/23547 (0%)]	Loss: 0.298838
Train Epoch: 4 [12800/23547 (54%)]	Loss: 0.292456
Train Epoch: 5 [0/23547 (0%)]	Loss: 0.286440
Train Epoch: 5 [12800/23547 (54%)]	Loss: 0.278135
Train Epoch: 6 [0/23547 (0%)]	Loss: 0.269713
Train Epoch: 6 [12800/23547 (54%)]	Loss: 0.250577
Train Epoch: 7 [0/23547 (0%)]	Loss: 0.234771
Train Epoch: 7 [12800/23547 (54%)]	Loss: 0.215353
Train Epoch: 8 [0/23547 (0%)]	Loss: 0.203740
Train Epoch: 8 [12800/23547 (54%)]	Loss: 0.197373
Train Epoch: 9 [0/23547 (0%)]	Loss: 0.189319
Train Epoch: 9 [12800/23547 (54%)]	Loss: 0.181829
Train Epoch: 10 [0/23547 (0%)]	Loss: 0.177804
Train Epoch: 10 [12800/23547 (54%)]	Loss: 0.174414


"\nfor data, _ in test_loader:\n    trans = transforms.RandomApply([AddGaussianNoise()], p=1)\n    data = trans(data.to(device))\n    test_output = decoder(encoder(data))\n    test_data = data * 0.5 + 0.5\n    test_output = test_output * 0.5 + 0.5\n    out_img = torch.squeeze(test_output.cpu().data)\n    print(out_img.size())\n    for i in range(2):\n        plt.subplot(1,2,1)\n        plt.imshow(torch.squeeze(test_data[i]).cpu().numpy(),cmap='gray')\n        plt.subplot(1,2,2)\n        plt.imshow(out_img[i].numpy(),cmap='gray')\n        plt.show()\n"